<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<br><br><br>
<h2><font color="#00586D" size=4>Módulo 2</font></h2>

<h1><font color="#00586D" size=5>Ejercicio SWAPI</font></h1>

<br><br><br>
<div style="text-align: right">
<font color="#00586D" size=3>Jacinto Arias</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>

#   Ejercicio SWAPI

**SWAPI** es una API pública creada para el aprendizaje de este tipo de tecnologías que contiene la información del universo de star wars.

Podemos encontrarla en https://swapi.dev/ y su API es https://swapi.dev/api/


##  Ejercicio 1

Navega la documentación de SWAPI y familiarizate con los endpoints `/people` y `/films`


In [ ]:
import requests

In [ ]:
url = "https://swapi.dev/api"

In [ ]:
requests.get(f"{url}/people").json()

In [ ]:
requests.get(f"{url}/people/1").json()

In [ ]:
requests.get(f"{url}/films").json()


##  Ejercicio 2

Crear una función `search_people(name)`, que utilizando SWAPI y el endpoint `people?search=<string>` devuelva los campos `name` y `url` de los resultados que encuentre.

In [ ]:
def search_people(name):
    response = requests.get(f"{url}/people?search={name}").json()['results']
    
    l = [ dict(name=person['name'], url=person['url']) for person in response ]
    
    return l

In [ ]:
search_people("skywalker")



## <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 3
 

Crear una función `get_user_films(id)` para SWAPI que dado el id de una persona devuelva el siguiente json:

```json
{
    "name": <name>,
    "films": [
        <title1>,
        <title2>,
        ...,
        <titlen>
    ]
}
```

**PISTA:** El id de un personaje viene indicado por un número tal y como se incluye en el capo `url` que nos devuelve la API.

**PISTA:** Básicamente la función debe llamar al endpoint `people/{id}` para obtener el nombre y las referencias a las películas para la persona indicada. Luego debe realizar (en un bucle) una llamada para cada endpoint de la película correspondiente y recuperar los títulos. Una vez realizado esto deberá componer el json final.

In [ ]:
def get_user_films(user_id):
    
    response = requests.get(f"{url}/people/{user_id}").json()
    
    name = response['name']
    films = [ requests.get(film).json()['title'] for film in response['films'] ]
    
    return {
        'name': name,
        'films': films
    }


In [ ]:
get_user_films(5)

## <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 4

Obten los nombres de los 50 primeros personajes que hay almacenados en la API.

__Pregunta__ ¿Cuántos valores devuelve la API?

__Respuesta__ Parece que la API devuelve cad vez un número limitado de elementos, esto se debe al factor de la __paginación__ que limita la cantidad de elementos que la API puede devolver cada vez y exige recuperar la información página a página. 

- En SWAPI la página se solicita con el queryparam `page`
- Si  respuesta de una consulta contiene el campo `previous` o `next`nos indica la existencia de páginas antes o despues de la llamada actual

In [ ]:
page = 0

In [ ]:
requests.get(f"{url}/people?page={page}").json()

In [ ]:
people = []

for page in range(1, 6):
    people += requests.get(f"{url}/people?page={page}").json()["results"]

In [ ]:
len(people)

In [ ]:
[p["name"] for p in people[:50]]

## <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 5

En este ejercicio crearemos una pequeña base de datos SQLite para almacenar las entidades `people` y `films` así como la relación entre ellas, donde un personaje ha podido participar en más de una pelicula y en una película aparecen más de un personaje.

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///swapi.db

In [ ]:
%sql PRAGMA foreign_keys = ON

Para poder hacer las insercciones podemos generar la query usando un string y luego incluirla en la consola de SQL. Por ejemplo

In [ ]:
%%sql

CREATE TABLE test (
    test_id INTEGER PRIMARY KEY,
    test_name VARCHAR(20)
)

Ahora hacemos los inserts

In [ ]:
query = "INSERT INTO test VALUES"

query += f"""({0}, "test")"""

Podemos lanzar una consulta desde una variable con `$`

In [ ]:
%sql $query

In [ ]:
%%sql

SELECT * FROM test

Recuerda que es posible incluir varios valores en un mismo insert, puedes crear un insert de muchos valores utilizando un bucle

### <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 5.1

Crea la tabla para la entidad `films`, incluye los siguientes campos

- `film_id`
- `episode_id` ue será la clave primaria y tomará el valor del campo `episode_id` que nos devuelve la API y que identifica a cada película
- `title`
- `director`

La clave primaria film_id tomará el valor númerico que se encuentra al final del campo `url` que nos devuelve la API y que identifica a cada película, para poder extraer este valor podemos separar la url utilizando `split` y obtener el valor en la posición correspondiente. Por ejemplo de la url `https://swapi.dev/api/films/1/` deberíamos obtener el valor `1`

Después realiza una llamada a la API e introduce todas las peliculas devueltas como filas de la tabla recién creada

In [ ]:
%%sql

CREATE TABLE films (
    film_id INTEGER PRIMARY KEY,
    episode_id INTEGER,
    title VARCHAR(20),
    director VARCHAR(20),
    release_date VARCHAR(20)
)

In [ ]:
def get_insert_film(films):
       
    query = f"""INSERT INTO films VALUES"""

    for i, f in enumerate(films):
        query += f"""({f['url'].split("/")[-2]}, {f['episode_id']}, "{f['title']}", "{f['director']}", "{f['release_date']}")""" + ","
    
    return query[:-1]

In [ ]:
films = requests.get(f"{url}/films").json()

In [ ]:
q = get_insert_film(films["results"])

In [ ]:
%sql $q

In [ ]:
%%sql

SELECT * FROM films

### <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 5.2

Crea la tabla para la entidad `people`, incluye los siguientes campos

- `person_id` 
- `name`
- `height`
- `mass`
- `hair_color`

La clave primaria person_id tomará el valor númerico que se encuentra al final del campo `url` que nos devuelve la API y que identifica a cada personaje, para poder extraer este valor podemos separar la url utilizando `split` y obtener el valor en la posición correspondiente. Por ejemplo de la url `https://swapi.dev/api/people/1/` deberíamos obtener el valor `1`

Una vez creada la tabla después realiza una llamada a la API e introduce todas las personas devueltas como filas de la tabla recién creada

In [ ]:
%sql DROP TABLE people

In [ ]:
%%sql

CREATE TABLE people (
    person_id INTEGER PRIMARY KEY,
    name VARCHAR(20),
    height INTEGER,
    mass INTEGER,
    hair_color VARCHAR(20)
)

In [ ]:
def get_insert_people(people):
       
    query = f"""INSERT INTO people VALUES"""

    for i, p in enumerate(people):
        query += f"""("{p['url'].split("/")[-2]}", "{p['name']}", "{p['height']}", "{p['mass']}", "{p['hair_color']}")""" + ","
    
    return query[:-1]

In [ ]:
people = requests.get(f"{url}/people").json()["results"]

In [ ]:
q = get_insert_people(people)

In [ ]:
%sql $q

In [ ]:
%%sql

SELECT * FROM people

### <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 5.3

Crea la tabla para la entidad `people_in_films` que modele la relación entre cada persona y las películas en las que aparece

__Pista:__ Habrá una fila para cada película y persona

Utiliza los datos obtenidos de la API para rellenar esta tabla

In [ ]:
%sql DROP TABLE people_in_films

In [ ]:
%%sql

CREATE TABLE people_in_films (
    person_id INTEGER,
    film_id INTEGER,
    
    FOREIGN KEY(person_id) REFERENCES people(person_id),
    FOREIGN KEY(film_id) REFERENCES films(film_id)
)

In [ ]:
def get_insert_film(people):
       
    query = f"""INSERT INTO people_in_films VALUES"""

    for i, p in enumerate(people):
        for f in p["films"]:
            query += f"""("{p['url'].split("/")[-2]}", "{f.split("/")[-2]}")""" + ","
    
    return query[:-1]

In [ ]:
q = get_insert_film(people)

In [ ]:
%sql $q

### <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 5.4

Realiza una consulta donde obtengamos el nombre y el título de cada persona y cada película

In [ ]:
%%sql res <<

SELECT 
    f.title,
    p.name
FROM films f
JOIN people_in_films pf ON f.film_id = pf.film_id
JOIN people p ON pf.person_id = p.person_id

In [ ]:
res.DataFrame().head()

### <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 5.5

Realiza una consulta donde obtengamos el total de personajes que aparecen en cada película

In [ ]:
%%sql res <<

SELECT 
    f.title,
    count(*) total_people
FROM films f
JOIN people_in_films pf ON f.film_id = pf.film_id
JOIN people p ON pf.person_id = p.person_id
GROUP BY f.title

In [ ]:
res.DataFrame().head()

### <i class="fa fa-pencil-square-o" aria-hidden="true"></i> Ejercicio 5.6

Realiza una consulta donde obtengamos el número de películas en las que aparece cada personaje

In [ ]:
%%sql res <<

SELECT 
    p.name,
    count(*) total_films
FROM films f
JOIN people_in_films pf ON f.film_id = pf.film_id
JOIN people p ON pf.person_id = p.person_id
GROUP BY p.name

In [ ]:
res.DataFrame().head()

## Ejercicio 6

En este ejercicio reproduciremos una API similar a SWAPI utilizando flask.

Para escribir un código de calidad configuraremos sqlite3 para que al realizar las consultas pueda recibir datos a modo de diccionario, como podemos comprobar en el siguiente código

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect("swapi.db")
con.row_factory = sqlite3.Row

In [ ]:
query = f"SELECT * FROM people LIMIT 10"

cur = con.cursor()

# Return all results of query
cur.execute(query)
data = cur.fetchall()
body = [dict(d) for d in data]


In [ ]:
body

### Ejercicio 6.1

Crea una API en flask con una función `/people` que liste el contenido de la tabla correspondiente en SQL

In [ ]:
from flask import Flask, jsonify, request

In [ ]:
app = Flask(__name__)

@app.route("/people", methods=['GET'])
def get_people():
    
    # Si ejecutamos esto dentro de una app independiente podemos reusar la conexi'on
    con = sqlite3.connect("swapi.db")
    con.row_factory = sqlite3.Row
    
    name = request.args.get("name", "")
    
    query = f"SELECT * FROM people WHERE name"
    
    cur = con.cursor()
    cur.execute(query)
    data = cur.fetchall()
    body = [dict(d) for d in data]
    
    return jsonify(body)

    # Si ejecutamos esto dentro de una app independiente podemos reusar la conexi'on
    con.close()

app.run(host="localhost")

### Ejercicio 6.2

Modifica la función anterior para que permita filtrar las personas por nombre usando un query string parameter llamado `name`

In [ ]:
app = Flask(__name__)

@app.route("/people", methods=['GET'])
def get_people():
    
    # Si ejecutamos esto dentro de una app independiente podemos reusar la conexi'on
    con = sqlite3.connect("swapi.db")
    con.row_factory = sqlite3.Row
    
    name = request.args.get("name", "")
    
    query = f"SELECT * FROM people WHERE name LIKE '%{name}%' LIMIT 10"
    
    cur = con.cursor()
    cur.execute(query)
    data = cur.fetchall()
    body = [dict(d) for d in data]
    
    return jsonify(body)

    # Si ejecutamos esto dentro de una app independiente podemos reusar la conexi'on
    con.close()

app.run(host="localhost")

### Ejercicio 6.3

Crea una API en flask con una función `/people/<person_id>:` que devuelva la persona con el id correspondiente de la tabla people

In [ ]:
app = Flask(__name__)

@app.route("/people/<person_id>", methods=['GET'])
def get_people_person_id(person_id):
    
    # Si ejecutamos esto dentro de una app independiente podemos reusar la conexi'on
    con = sqlite3.connect("swapi.db")
    con.row_factory = sqlite3.Row
    
    query = f"SELECT * FROM people WHERE person_id = {person_id}"
    
    cur = con.cursor()
    cur.execute(query)
    data = cur.fetchall()
    body = [dict(d) for d in data]
    
    return jsonify(body)

    # Si ejecutamos esto dentro de una app independiente podemos reusar la conexi'on
    con.close()

app.run(host="localhost")

### Ejercicio 6.4

Crea una API en flask con una función `/people/<perosn_id>/films` que liste las películas para la persona correspondiente

In [ ]:
app = Flask(__name__)

@app.route("/people/<person_id>/films", methods=['GET'])
def get_people_person_id_films(person_id):
    
    # Si ejecutamos esto dentro de una app independiente podemos reusar la conexi'on
    con = sqlite3.connect("swapi.db")
    con.row_factory = sqlite3.Row
    
    query = f"""
        SELECT 
            f.film_id,
            f.episode_id,
            f.title
        FROM films f
        JOIN people_in_films pf ON f.film_id = pf.film_id
        JOIN people p ON pf.person_id = p.person_id AND p.person_id = {person_id}
    """
    
    cur = con.cursor()
    cur.execute(query)
    data = cur.fetchall()
    body = [dict(d) for d in data]
    
    return jsonify(body)

    # Si ejecutamos esto dentro de una app independiente podemos reusar la conexi'on
    con.close()

app.run(host="localhost")